In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline  

from sklearn import metrics

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
df_train=pd.read_csv('train.csv')
df_train.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s17_av,s17_std,s18_av,s18_std,s19_av,s19_std,s20_av,s20_std,s21_av,s21_std
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,392.000000,0.000000,2388.0,0.0,100.0,0.0,39.060000,0.000000,23.419000,0.000000
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,392.000000,0.000000,2388.0,0.0,100.0,0.0,39.030000,0.042426,23.421300,0.003253
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,391.333333,1.154701,2388.0,0.0,100.0,0.0,39.003333,0.055076,23.395600,0.044573
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,391.500000,1.000000,2388.0,0.0,100.0,0.0,38.972500,0.076322,23.390175,0.037977
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,391.800000,1.095445,2388.0,0.0,100.0,0.0,38.958000,0.073621,23.393020,0.033498


In [3]:
df_test=pd.read_csv('test.csv')
print(df_test.shape)
df_test.head()

(13096, 71)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s17_av,s17_std,s18_av,s18_std,s19_av,s19_std,s20_av,s20_std,s21_av,s21_std
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,392.000000,0.000000,2388.0,0.0,100.0,0.0,38.860000,0.000000,23.37350,0.000000
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,392.500000,0.707107,2388.0,0.0,100.0,0.0,38.940000,0.113137,23.38255,0.012799
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,392.666667,0.577350,2388.0,0.0,100.0,0.0,38.986667,0.113725,23.39390,0.021642
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,392.250000,0.957427,2388.0,0.0,100.0,0.0,38.990000,0.093095,23.38885,0.020353
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,391.800000,1.303840,2388.0,0.0,100.0,0.0,38.990000,0.080623,23.39368,0.020672


In [4]:
df_test.label_bnc.sum()

332

In [5]:
# original features
features_orig = ['setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']

# original + extracted fetures
features_extr =[u'setting1', u'setting2', u'setting3', u's1', u's2',
       u's3', u's4', u's5', u's6', u's7', u's8', u's9', u's10', u's11', u's12',
       u's13', u's14', u's15', u's16', u's17', u's18', u's19', u's20', u's21',
       u's1_av', u's1_std', u's2_av',
       u's2_std', u's3_av', u's3_std', u's4_av', u's4_std', u's5_av',
       u's5_std', u's6_av', u's6_std', u's7_av', u's7_std', u's8_av',
       u's8_std', u's9_av', u's9_std', u's10_av', u's10_std', u's11_av',
       u's11_std', u's12_av', u's12_std', u's13_av', u's13_std', u's14_av',
       u's14_std', u's15_av', u's15_std', u's16_av', u's16_std', u's17_av',
       u's17_std', u's18_av', u's18_std', u's19_av', u's19_std', u's20_av',
       u's20_std', u's21_av', u's21_std']

imp_features=['s9_av', 's8_av', 's3_av', 's17_av', 's11_av', 's7_av', 's15_av', 's20_av', 's21_av', 's2_av']

y_train = df_train['label_bnc']
y_test = df_test['label_bnc']

In [6]:
y_train.sum()

3100

## parameter tunning

In [7]:
def bin_classify(model, clf, features, params=None, score=None):
    
    """Perfor Grid Search hyper parameter tuning on a classifier.
    
    Args:
        model (str): The model name identifier
        clf (clssifier object): The classifier to be tuned
        features (list): The set of input features names
        params (dict): Grid Search parameters
        score (str): Grid Search score
        
    Returns:
        Tuned Clssifier object
        dataframe of model predictions and scores
    
    """
    
    
    X_train = df_train[features]

    X_test = df_test[features] 
    
    grid_search = model_selection.GridSearchCV(estimator=clf, param_grid=params, cv=5, scoring=score, n_jobs=-1)

    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    
    if hasattr(grid_search, 'predict_proba'):   
        y_score = grid_search.predict_proba(X_test)[:,1]
    elif hasattr(grid_search, 'decision_function'):
        y_score = grid_search.decision_function(X_test)
    else:
        y_score = y_pred
        
    predictions = {'y_pred' : y_pred, 'y_score' : y_score}
    df_predictions = pd.DataFrame.from_dict(predictions)
    
    return grid_search.best_estimator_, df_predictions

In [8]:
def bin_class_metrics(model, y_test, y_pred, y_score, print_out=True):
    
    """Calculate main binary classifcation metrics, plot AUC ROC and Precision-Recall curves.
    
    Args:
        model (str): The model name identifier
        y_test (series): Contains the test label values
        y_pred (series): Contains the predicted values
        y_score (series): Contains the predicted scores
        print_out (bool): Print the classification metrics and thresholds values
        plot_out (bool): Plot AUC ROC, Precision-Recall, and Threshold curves
        
    Returns:
        dataframe: The combined metrics in single dataframe
        dataframe: ROC thresholds
        dataframe: Precision-Recall thresholds
        Plot: AUC ROC
        plot: Precision-Recall
        plot: Precision-Recall threshold; also show the number of engines predicted for maintenace per period (queue).
        plot: TPR-FPR threshold
        
    """
      
    binclass_metrics = {
                        'Accuracy' : metrics.accuracy_score(y_test, y_pred),
                        'Precision' : metrics.precision_score(y_test, y_pred),
                        'Recall' : metrics.recall_score(y_test, y_pred),
                        'F1 Score' : metrics.f1_score(y_test, y_pred),
                        'ROC AUC' : metrics.roc_auc_score(y_test, y_score)
                       }

    df_metrics = pd.DataFrame.from_dict(binclass_metrics, orient='index')
    df_metrics.columns = [model]  

    if print_out:
        print('-----------------------------------------------------------')
        print(model, '\n')
        print('Confusion Matrix:')
        print(metrics.confusion_matrix(y_test, y_pred))
        print('\nClassification Report:')
        print(metrics.classification_report(y_test, y_pred))
        print('\nMetrics:')
        print(df_metrics)

    return  df_metrics


**A** after feature extraction features=features_extr<br>
**B** original features features=features_orig<br>
**I** important features=imp_features

In [9]:
%%time
model = 'Logistic Regression B'
clf_lgrb = LogisticRegression(random_state=123)
gs_params = {'C': [.01, 0.1, 1.0, 10], 'solver': ['liblinear', 'lbfgs']}
gs_score = 'roc_auc'

clf_lgrb, pred_lgrb = bin_classify(model, clf_lgrb, features_orig, params=gs_params, score=gs_score)
print ('\nBest Parameters:\n',clf_lgrb)

metrics_lgrb= bin_class_metrics(model, y_test, pred_lgrb.y_pred, pred_lgrb.y_score, print_out=True)


Best Parameters:
 LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)
-----------------------------------------------------------
Logistic Regression B 

Confusion Matrix:
[[12713    51]
 [  141   191]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.79      0.58      0.67       332

avg / total       0.98      0.99      0.98     13096


Metrics:
           Logistic Regression B
Accuracy                0.985339
Precision               0.789256
Recall                  0.575301
F1 Score                0.665505
ROC AUC                 0.984889
Wall time: 7.16 s


In [10]:
%%time
model = 'Logistic Regression A'
clf_lgrb = LogisticRegression(random_state=123)
gs_params = {'C': [.01, 0.1, 1.0, 10], 'solver': ['liblinear', 'lbfgs']}
gs_score = 'roc_auc'

clf_lgrb, pred_lgrb = bin_classify(model, clf_lgrb, features_extr, params=gs_params, score=gs_score)
print ('\nBest Parameters:\n',clf_lgrb)

metrics_lgra= bin_class_metrics(model, y_test, pred_lgrb.y_pred, pred_lgrb.y_score, print_out=True)


Best Parameters:
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)
-----------------------------------------------------------
Logistic Regression A 

Confusion Matrix:
[[12730    34]
 [  128   204]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.86      0.61      0.72       332

avg / total       0.99      0.99      0.99     13096


Metrics:
           Logistic Regression A
Accuracy                0.987630
Precision               0.857143
Recall                  0.614458
F1 Score                0.715789
ROC AUC                 0.986305
Wall time: 9.29 s


In [11]:
%%time
model = 'Logistic Regression I'
clf_lgrb = LogisticRegression(random_state=123)
gs_params = {'C': [.01, 0.1, 1.0, 10], 'solver': ['liblinear', 'lbfgs']}
gs_score = 'roc_auc'

clf_lgrb, pred_lgrb = bin_classify(model, clf_lgrb, imp_features, params=gs_params, score=gs_score)
print ('\nBest Parameters:\n',clf_lgrb)

metrics_lgri= bin_class_metrics(model, y_test, pred_lgrb.y_pred, pred_lgrb.y_score, print_out=True)


Best Parameters:
 LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)
-----------------------------------------------------------
Logistic Regression I 

Confusion Matrix:
[[12713    51]
 [  144   188]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.79      0.57      0.66       332

avg / total       0.98      0.99      0.98     13096


Metrics:
           Logistic Regression I
Accuracy                0.985110
Precision               0.786611
Recall                  0.566265
F1 Score                0.658494
ROC AUC                 0.987483
Wall time: 8.47 s


In [12]:
metrics_lgr = pd.concat([metrics_lgrb, metrics_lgra,metrics_lgri], axis=1)
metrics_lgr

,Logistic Regression B,Logistic Regression A,Logistic Regression I
Accuracy,0.985339,0.987630,0.985110
Precision,0.789256,0.857143,0.786611
Recall,0.575301,0.614458,0.566265
F1 Score,0.665505,0.715789,0.658494
ROC AUC,0.984889,0.986305,0.987483


In [13]:
%%time
model = 'Decision Tree B'
clf_dtrb = DecisionTreeClassifier(random_state=123)
gs_params = {'max_depth': [2, 3, 4, 5, 6], 'criterion': ['gini', 'entropy']}
gs_score = 'roc_auc'

clf_dtrb, pred_dtrb = bin_classify(model, clf_dtrb, features_orig, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_dtrb)

metrics_dtrb= bin_class_metrics(model, y_test, pred_dtrb.y_pred, pred_dtrb.y_score, print_out=True)


Best Parameters:
 DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')
-----------------------------------------------------------
Decision Tree B 

Confusion Matrix:
[[12720    44]
 [  162   170]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.79      0.51      0.62       332

avg / total       0.98      0.98      0.98     13096


Metrics:
           Decision Tree B
Accuracy          0.984270
Precision         0.794393
Recall            0.512048
F1 Score          0.622711
ROC AUC           0.972794
Wall time: 4.55 s


In [14]:
%%time
model = 'Decision Tree A'
clf_dtrb = DecisionTreeClassifier(random_state=123)
gs_params = {'max_depth': [2, 3, 4, 5, 6], 'criterion': ['gini', 'entropy']}
gs_score = 'roc_auc'

clf_dtrb, pred_dtrb = bin_classify(model, clf_dtrb, features_extr, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_dtrb)

metrics_dtra= bin_class_metrics(model, y_test, pred_dtrb.y_pred, pred_dtrb.y_score, print_out=True)


Best Parameters:
 DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')
-----------------------------------------------------------
Decision Tree A 

Confusion Matrix:
[[12675    89]
 [  123   209]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.99      0.99     12764
          1       0.70      0.63      0.66       332

avg / total       0.98      0.98      0.98     13096


Metrics:
           Decision Tree A
Accuracy          0.983812
Precision         0.701342
Recall            0.629518
F1 Score          0.663492
ROC AUC           0.964234
Wall time: 7.66 s


In [15]:
%%time
model = 'Decision Tree I'
clf_dtrb = DecisionTreeClassifier(random_state=123)
gs_params = {'max_depth': [2, 3, 4, 5, 6], 'criterion': ['gini', 'entropy']}
gs_score = 'roc_auc'

clf_dtrb, pred_dtrb = bin_classify(model, clf_dtrb, imp_features, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_dtrb)

metrics_dtri= bin_class_metrics(model, y_test, pred_dtrb.y_pred, pred_dtrb.y_score, print_out=True)


Best Parameters:
 DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')
-----------------------------------------------------------
Decision Tree I 

Confusion Matrix:
[[12677    87]
 [  129   203]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.99      0.99     12764
          1       0.70      0.61      0.65       332

avg / total       0.98      0.98      0.98     13096


Metrics:
           Decision Tree I
Accuracy          0.983506
Precision         0.700000
Recall            0.611446
F1 Score          0.652733
ROC AUC           0.965563
Wall time: 4.08 s


In [16]:
metrics_dtr = pd.concat([metrics_dtrb, metrics_dtra,metrics_dtri], axis=1)
metrics_dtr

,Decision Tree B,Decision Tree A,Decision Tree I
Accuracy,0.984270,0.983812,0.983506
Precision,0.794393,0.701342,0.700000
Recall,0.512048,0.629518,0.611446
F1 Score,0.622711,0.663492,0.652733
ROC AUC,0.972794,0.964234,0.965563


In [17]:
%%time
model = 'Random Forest B'
clf_rfcb = RandomForestClassifier(n_estimators=50, random_state=123)
gs_params = {'max_depth': [4, 5, 6, 7, 8], 'criterion': ['gini', 'entropy']}
gs_score = 'roc_auc'

clf_rfcb, pred_rfcb = bin_classify(model, clf_rfcb, features_orig, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_rfcb)

metrics_rfcb= bin_class_metrics(model, y_test, pred_rfcb.y_pred, pred_rfcb.y_score, print_out=True)


Best Parameters:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)
-----------------------------------------------------------
Random Forest B 

Confusion Matrix:
[[12725    39]
 [  134   198]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.84      0.60      0.70       332

avg / total       0.99      0.99      0.99     13096


Metrics:
           Random Forest B
Accuracy          0.986790
Precision         0.835443
Recall            0.596386
F1 Score          0.695958
ROC AUC           0.990418
Wall time: 16.3 s


In [18]:
%%time
model = 'Random Forest A'
clf_rfcb = RandomForestClassifier(n_estimators=50, random_state=123)
gs_params = {'max_depth': [4, 5, 6, 7, 8], 'criterion': ['gini', 'entropy']}
gs_score = 'roc_auc'

clf_rfcb, pred_rfcb = bin_classify(model, clf_rfcb, features_extr, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_rfcb)

metrics_rfca= bin_class_metrics(model, y_test, pred_rfcb.y_pred, pred_rfcb.y_score, print_out=True)


Best Parameters:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)
-----------------------------------------------------------
Random Forest A 

Confusion Matrix:
[[12721    43]
 [  140   192]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.82      0.58      0.68       332

avg / total       0.98      0.99      0.98     13096


Metrics:
           Random Forest A
Accuracy          0.986026
Precision         0.817021
Recall            0.578313
F1 Score          0.677249
ROC AUC           0.989041
Wall time: 29.1 s


In [19]:
%%time
model = 'Random Forest I'
clf_rfcb = RandomForestClassifier(n_estimators=50, random_state=123)
gs_params = {'max_depth': [4, 5, 6, 7, 8], 'criterion': ['gini', 'entropy']}
gs_score = 'roc_auc'

clf_rfcb, pred_rfcb = bin_classify(model, clf_rfcb, imp_features, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_rfcb)

metrics_rfci= bin_class_metrics(model, y_test, pred_rfcb.y_pred, pred_rfcb.y_score, print_out=True)


Best Parameters:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)
-----------------------------------------------------------
Random Forest I 

Confusion Matrix:
[[12718    46]
 [  146   186]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.80      0.56      0.66       332

avg / total       0.98      0.99      0.98     13096


Metrics:
           Random Forest I
Accuracy          0.985339
Precision         0.801724
Recall            0.560241
F1 Score          0.659574
ROC AUC           0.986600
Wall time: 21.1 s


In [20]:
metrics_rfc = pd.concat([metrics_rfcb, metrics_rfca,metrics_rfci], axis=1)
metrics_rfc

,Random Forest B,Random Forest A,Random Forest I
Accuracy,0.986790,0.986026,0.985339
Precision,0.835443,0.817021,0.801724
Recall,0.596386,0.578313,0.560241
F1 Score,0.695958,0.677249,0.659574
ROC AUC,0.990418,0.989041,0.986600


In [21]:
%%time
model = 'KNN B'
clf_knnb = KNeighborsClassifier(n_jobs=-1)
gs_params = {'n_neighbors': [9, 10, 11, 12, 13]}
gs_score = 'roc_auc'

clf_knnb, pred_knnb = bin_classify(model, clf_knnb, features_orig, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_knnb)

metrics_knnb= bin_class_metrics(model, y_test, pred_knnb.y_pred, pred_knnb.y_score, print_out=True)


Best Parameters:
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=13, p=2,
           weights='uniform')
-----------------------------------------------------------
KNN B 

Confusion Matrix:
[[12704    60]
 [  158   174]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.74      0.52      0.61       332

avg / total       0.98      0.98      0.98     13096


Metrics:
              KNN B
Accuracy   0.983354
Precision  0.743590
Recall     0.524096
F1 Score   0.614841
ROC AUC    0.939173
Wall time: 10.5 s


In [22]:
%%time
model = 'KNN A'
clf_knnb = KNeighborsClassifier(n_jobs=-1)
gs_params = {'n_neighbors': [9, 10, 11, 12, 13]}
gs_score = 'roc_auc'

clf_knnb, pred_knnb = bin_classify(model, clf_knnb, features_extr, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_knnb)

metrics_knna= bin_class_metrics(model, y_test, pred_knnb.y_pred, pred_knnb.y_score, print_out=True)


Best Parameters:
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=13, p=2,
           weights='uniform')
-----------------------------------------------------------
KNN A 

Confusion Matrix:
[[12713    51]
 [  145   187]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.79      0.56      0.66       332

avg / total       0.98      0.99      0.98     13096


Metrics:
              KNN A
Accuracy   0.985034
Precision  0.785714
Recall     0.563253
F1 Score   0.656140
ROC AUC    0.927888
Wall time: 46.6 s


In [23]:
%%time
model = 'KNN I'
clf_knnb = KNeighborsClassifier(n_jobs=-1)
gs_params = {'n_neighbors': [9, 10, 11, 12, 13]}
gs_score = 'roc_auc'

clf_knnb, pred_knnb = bin_classify(model, clf_knnb, imp_features, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_knnb)

metrics_knni= bin_class_metrics(model, y_test, pred_knnb.y_pred, pred_knnb.y_score, print_out=True)


Best Parameters:
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=13, p=2,
           weights='uniform')
-----------------------------------------------------------
KNN I 

Confusion Matrix:
[[12702    62]
 [  154   178]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     12764
          1       0.74      0.54      0.62       332

avg / total       0.98      0.98      0.98     13096


Metrics:
              KNN I
Accuracy   0.983506
Precision  0.741667
Recall     0.536145
F1 Score   0.622378
ROC AUC    0.919352
Wall time: 5.28 s


In [24]:
metrics_knn = pd.concat([metrics_knnb, metrics_knna,metrics_knni], axis=1)
metrics_knn

,KNN B,KNN A,KNN I
Accuracy,0.983354,0.985034,0.983506
Precision,0.743590,0.785714,0.741667
Recall,0.524096,0.563253,0.536145
F1 Score,0.614841,0.656140,0.622378
ROC AUC,0.939173,0.927888,0.919352


In [25]:
%%time
model = 'Gaussian NB B'
clf_gnbb = GaussianNB()
gs_params = {} 
gs_score = 'roc_auc'

clf_gnbb, pred_gnbb = bin_classify(model, clf_gnbb, features_orig, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_gnbb)

metrics_gnbb= bin_class_metrics(model, y_test, pred_gnbb.y_pred, pred_gnbb.y_score, print_out=True)


Best Parameters:
 GaussianNB(priors=None)
-----------------------------------------------------------
Gaussian NB B 

Confusion Matrix:
[[12465   299]
 [   40   292]]

Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.98      0.99     12764
          1       0.49      0.88      0.63       332

avg / total       0.98      0.97      0.98     13096


Metrics:
           Gaussian NB B
Accuracy        0.974114
Precision       0.494078
Recall          0.879518
F1 Score        0.632719
ROC AUC         0.991251
Wall time: 2.91 s


In [26]:
%%time
model = 'Gaussian NB A'
clf_gnbb = GaussianNB()
gs_params = {} 
gs_score = 'roc_auc'

clf_gnbb, pred_gnbb = bin_classify(model, clf_gnbb, features_extr, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_gnbb)

metrics_gnba= bin_class_metrics(model, y_test, pred_gnbb.y_pred, pred_gnbb.y_score, print_out=True)


Best Parameters:
 GaussianNB(priors=None)
-----------------------------------------------------------
Gaussian NB A 

Confusion Matrix:
[[12470   294]
 [   44   288]]

Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.98      0.99     12764
          1       0.49      0.87      0.63       332

avg / total       0.98      0.97      0.98     13096


Metrics:
           Gaussian NB A
Accuracy        0.974191
Precision       0.494845
Recall          0.867470
F1 Score        0.630197
ROC AUC         0.990313
Wall time: 2.96 s


In [27]:
%%time
model = 'Gaussian NB I'
clf_gnbb = GaussianNB()
gs_params = {} 
gs_score = 'roc_auc'

clf_gnbb, pred_gnbb = bin_classify(model, clf_gnbb, imp_features, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_gnbb)

metrics_gnbi= bin_class_metrics(model, y_test, pred_gnbb.y_pred, pred_gnbb.y_score, print_out=True)


Best Parameters:
 GaussianNB(priors=None)
-----------------------------------------------------------
Gaussian NB I 

Confusion Matrix:
[[12505   259]
 [   64   268]]

Classification Report:
             precision    recall  f1-score   support

          0       0.99      0.98      0.99     12764
          1       0.51      0.81      0.62       332

avg / total       0.98      0.98      0.98     13096


Metrics:
           Gaussian NB I
Accuracy        0.975336
Precision       0.508539
Recall          0.807229
F1 Score        0.623981
ROC AUC         0.988285
Wall time: 2.86 s


In [28]:
metrics_gnb = pd.concat([metrics_gnbb, metrics_gnba,metrics_gnbi], axis=1)
metrics_gnb

,Gaussian NB B,Gaussian NB A,Gaussian NB I
Accuracy,0.974114,0.974191,0.975336
Precision,0.494078,0.494845,0.508539
Recall,0.879518,0.867470,0.807229
F1 Score,0.632719,0.630197,0.623981
ROC AUC,0.991251,0.990313,0.988285
